**1. Cargar librerías**

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

**2. Cargar datos**

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("karimabdulnabi/fruit-classification10-class")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fruit-classification10-class' dataset.
Path to dataset files: /kaggle/input/fruit-classification10-class


In [3]:
!pip install kaggle
from google.colab import files

In [ ]:
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d karimabdulnabi/fruit-classification10-class

In [ ]:
import zipfile

# Ruta al archivo ZIP descargado
zip_file_path = '/content/DATASET FRUIT.zip'

# Ruta a la carpeta donde deseas descomprimir el archivo
extracted_folder_path = '/content/Dataset'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [ ]:
imagenes = []
etiquetas = []

In [ ]:
import os
import glob
import cv2
import numpy as np

# Clear existing data to prevent accumulation on re-run
imagenes = []
etiquetas = []

# Base path to the dataset
base_dataset_path = '/content/Dataset/DATASET FRUIT'

# Define the directories containing the images (e.g., TRAIN, TEST)
data_subdirs = ['TRAIN', 'TEST']

# Map class names to integer labels
class_names = []

# Iterate through each data subdirectory (TRAIN, TEST)
current_label = 0
for data_subdir in data_subdirs:
    full_data_path = os.path.join(base_dataset_path, data_subdir)

    # Get all class subdirectories within TRAIN or TEST
    class_subdirs = [d for d in os.listdir(full_data_path) if os.path.isdir(os.path.join(full_data_path, d))]

    for class_name_full in class_subdirs:
        # Extract a cleaner class name, e.g., 'Strawberry' from 'Strawberry 224x224 TR'
        clean_class_name = class_name_full.split(' ')[0] # Takes 'Strawberry' from 'Strawberry 224x224 TR'

        # Assign a new label if this class hasn't been seen before
        if clean_class_name not in class_names:
            class_names.append(clean_class_name)

        # Get the integer label for this class
        label = class_names.index(clean_class_name)

        class_path = os.path.join(full_data_path, class_name_full)
        print(f'Processing images in: {class_path} (Label: {label}, Class: {clean_class_name})')

        # Iterate through all JPEG images in the current class directory
        for image_path in glob.iglob(f'{class_path}/*.jpeg'):
            test_image = cv2.imread(image_path)
            if test_image is not None:
                img_resized = cv2.resize(test_image, (224, 224))
                arreglo_imagen = np.array(img_resized)
                imagenes.append(arreglo_imagen)
                etiquetas.append(label)
            else:#
                print(f"Warning: Could not load image {image_path}")

print(f"Total images loaded: {len(imagenes)}")
print(f"Total labels loaded: {len(etiquetas)}")
print(f"Detected classes: {class_names}")

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np # Import numpy if not already imported in this scope

images = np.array(imagenes)
etiquetas = np.array(etiquetas)

train_images, test_images, train_labels, test_labels = train_test_split(images, etiquetas, test_size=0.2, random_state=42)

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
etiquetas = np.array(etiquetas)

In [ ]:
images = np.array(imagenes)

In [ ]:
np.shape(images)

In [ ]:
len(etiquetas)

In [ ]:
etiquetas[-1]

In [ ]:
plt.imshow(imagenes[0])

In [ ]:

# Convierte la imagen en un arreglo de numpy
arreglo_imagen = np.array(img_resized)
imagenes.append(arreglo_imagen)

In [ ]:

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# The class_names are already defined in cell PYzEy3_8J8gh
# We will reuse that variable for consistency and correct labels.
# If you run this cell independently, ensure `class_names` is populated correctly
# from the image loading step.


In [ ]:
plt.figure(figsize=(10,10))
for i in range(len(images)):
    if i >= 25: # Only plot the first 25 images to avoid excessive output
        break
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i])
    # Ensure the label index is within the bounds of class_names
    if etiquetas[i] < len(class_names):
        plt.xlabel(class_names[etiquetas[i]])
    else:
        plt.xlabel(f'Unknown Label: {etiquetas[i]}')
plt.show()

**3. Model Creation**

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(layers.Dropout(0.5)) # Add dropout layer with 50% dropout rate
model.add(layers.Dense(len(class_names)))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

**4. Evaluate the Model**

In [ ]:
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)

In [ ]:
predictions[0]

In [ ]:
import cv2 # Ensure cv2 is imported for cvtColor

def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  # Convert the float64 image (0-1) to uint8 (0-255) for cv2.cvtColor
  img_uint8 = (img * 255).astype(np.uint8)
  # Convert BGR to RGB for correct color display with matplotlib
  img_rgb = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2RGB)
  plt.imshow(img_rgb)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(len(predictions_array)))
  plt.yticks([])
  thisplot = plt.bar(range(len(predictions_array)), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('green')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 150
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 23
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 84
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 93
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 3
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 180
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()